In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## libraries

In [ ]:
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor, LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
import kaggle_evaluation.mcts_inference_server
seed=42
np.random.seed(seed)


## Read Data 

In [ ]:
train_path = "/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"
train_data = pd.read_csv(train_path)
train_data

## Cleaning 

1. drop column have same values or 90%
2. drop column dont have any meaning or not found on test ["Id", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"]
3. search about outliers or null values 

In [2]:
# select columns with 90% or more identical values and columns with all null values
def drop_column_name(df):
    drop_columns = []
    for column in df.columns:
        if df[column].value_counts().max() / len(df) >= 0.90 or df[column].isnull().all():
            drop_columns.append(column)
    return drop_columns

In [ ]:
drop_column = drop_column_name(train_data)
#add Column dont have any meaning whene training or not found on test dataset
drop_column.extend(["Id", "EnglishRules", "LudRules", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"])